In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd

from keras.utils.np_utils import to_categorical

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Input, Embedding, LSTM, Dense,Dropout,GRU,Masking
from keras.models import Model
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

from gensim.models import Word2Vec

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix


Using TensorFlow backend.
C:\ProgramData\Anaconda2\envs\deeplearning\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = pd.concat([train, test])

In [3]:

y_train = pd.read_csv("train.csv")
y_train.drop('User_ID', inplace=True, axis=1)
y_train.drop('Browser_Used', inplace=True, axis=1)
y_train.drop('Device_Used', inplace=True, axis=1)
y_train.drop('Description',inplace = True,axis=1)

train.drop('User_ID', inplace=True, axis=1)
train.drop('Browser_Used', inplace=True, axis=1)
train.drop('Device_Used', inplace=True, axis=1)
train.drop('Is_Response', inplace=True, axis=1)
train.head()



,Description
0,The room was kind of clean but had a VERY stro...
1,I stayed at the Crown Plaza April -- - April -...
2,I booked this hotel through Hotwire at the low...
3,Stayed here with husband and sons on the way t...
4,My girlfriends and I stayed here to celebrate ...


In [4]:
train = np.array(train['Description'])
for i in range(0,68336):
    train[i] = train[i].lower()

tokenizer = RegexpTokenizer(r'\w+')
for i in range(0,68336):
    train[i] = tokenizer.tokenize(train[i])


In [6]:
model99 = Word2Vec(train,size =100,window = 5,min_count =1)

In [7]:

c=np.zeros([68336,400,100])

In [8]:

b = np.zeros([100])
for i in range(0,68336):
    a = model99.wv[train[i]]
    if(len(a) >400):
        a = a[0:400]
    for j in range(0,(400-len(a))):
        a = np.vstack([a,b])
    c[i] = a
del model99
del a 
del b
del train

In [9]:
y_train = pd.get_dummies(y_train,columns=['Is_Response'])
x_train = c[0:38932]
x_test = c[38932:]
y_train = y_train['Is_Response_happy']
del c

In [165]:
main_input = Input(shape = (400,100))
mask=Masking(mask_value=0.0)(main_input)
lstm_out = LSTM(256)(mask)
out=Dense(64)(lstm_out)
output = Dense(1,activation = 'sigmoid')(out)
model = Model(inputs=main_input,outputs=output)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 400, 100)          0         
_________________________________________________________________
masking_16 (Masking)         (None, 400, 100)          0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 256)               365568    
_________________________________________________________________
dense_26 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 65        
Total params: 382,081
Trainable params: 382,081
Non-trainable params: 0
_________________________________________________________________


In [166]:
model.fit(x_train, y_train,batch_size=64,epochs= 4)

Epoch 1/4
38932/38932 [==============================] - 3310s 85ms/step - loss: 0.4055 - acc: 0.8185
Epoch 2/4
38932/38932 [==============================] - 3251s 84ms/step - loss: 0.2916 - acc: 0.8803
Epoch 3/4
38932/38932 [==============================] - 3263s 84ms/step - loss: 0.2586 - acc: 0.8953
Epoch 4/4
38932/38932 [==============================] - 3242s 83ms/step - loss: 0.2343 - acc: 0.9059


In [167]:
main_input2 = Input(shape = (400,100))
mask2=Masking(mask_value=0.0)(main_input2)
lstm_out2 = LSTM(256,weights = model.layers[2].get_weights())(mask2)
out2=Dense(64,weights = model.layers[3].get_weights())(lstm_out2)
model2 = Model(inputs=main_input2,outputs=out2)

train_transformed = model2.predict(x_train)
test_transformed = model2.predict(x_test)

other_data = pd.read_csv("Kish_train.csv")
other_data = np.array(other_data)

train_transformed = np.concatenate([train_transformed,other_data],axis=1)

train_xgb = train_transformed[0:37842]
valid_xgb = train_transformed[37842:]

y_train_xgb = y_train[0:37842]
y_valid_xgb = y_train[37842:]


array([[301,  40],
       [ 39, 710]], dtype=int64)

In [208]:
main_input3 = Input(shape = (73,))
drop0 = Dropout(0.2)(main_input3)
deninp1=Dense(100)(drop0)
drop1 = Dropout(0.5)(deninp1)
deninp2=Dense(20)(drop1)
output3 = Dense(1,activation = 'sigmoid')(deninp2)
model3 = Model(inputs=main_input3,outputs=output3)
model3.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 73)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 73)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 100)               7400      
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 20)                2020      
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 21        
Total params: 9,441
Trainable params: 9,441
Non-trainable params: 0
_________________________________________________________________


In [212]:
model3.fit(train_xgb,y_train_xgb,batch_size=64,epochs= 2000,validation_data=[valid_xgb,y_valid_xgb])

In [213]:
result = model3.predict(test_transformed)

In [183]:
np.savetxt(X=result,fname="result_with_masking_v15.csv")

(29404, 73)